# Visualizaciones de Géneros en Plataformas de Streaming
## Utilizo de plataforma web para las visualizaciones con Race Bar Charts

1. [Visualización de géneros de series de Netflix](https://public.flourish.studio/visualisation/18025904/)
2. [Visualización de géneros de películas de Netflix](https://public.flourish.studio/visualisation/18023613/)
3. [Visualización de géneros de series de Hulu](https://public.flourish.studio/visualisation/18025890/)
4. [Visualización de géneros de películas de Hulu](https://public.flourish.studio/visualisation/18025874/)
5. [Visualización de géneros de películas de Disney+](https://public.flourish.studio/visualisation/18025793/)


# Modifica del conjunto de datos para subirlo en *Flourish.studio*

Como primero se van a incluir todas las librerías necesarias

In [120]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly as px

# 1. Estudio preliminar de los datos

In [121]:
df_netflix = pd.read_csv('netflix_titles.csv')
df_prime = pd.read_csv('amazon_prime_titles.csv')
df_disney = pd.read_csv('disney_plus_titles.csv')
df_hulu = pd.read_csv('hulu_titles.csv')

### Exploracion general de los datos

In [122]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [123]:
df_netflix['type'].unique()

array(['Movie', 'TV Show'], dtype=object)

In [124]:
df_prime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [125]:
df_disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [126]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


### Los datos de las plataformas Netflix y Amazon prime son mas largos que los de hulu y Disney.

### Pero se subraya el numero de "Not a Number" en la columna "date_added" para el dataset de Amazon prime. Hay solo 155 valores validos, entonces se va a excluir este dataset del analisis

In [127]:
# Estudio de las columnas en las bases de datos
columnas_netflix = set(df_netflix.columns)
columnas_disney = set(df_disney.columns)
columnas_hulu = set(df_hulu.columns)

# Se comprueban las columnas en comun
columnas_comun = columnas_netflix & columnas_disney & columnas_hulu

# Fundamental es tener en comun las columnas "date_added", "listed_in", "type"
print(columnas_comun)

{'listed_in', 'show_id', 'title', 'director', 'release_year', 'country', 'description', 'rating', 'duration', 'date_added', 'type', 'cast'}


## El siguiente serà un trabajo para desarrollo de una funcion general para cambiar el formato de los datos y alcanzar en tener un dataset que puede ser subido en flourish.com para el utilizo de la herramienta de visualizacion animata

# 2. DESARROLLO FUNCION

Funcion para peliculas

In [128]:
def modifica_datos_movies(df) :
    # Se divide entre datos sobre movies y datos sobre series tv
    df_movies = df[df['type']=='Movie']
    df_movies.reset_index(drop=True, inplace=True)

    # Se convierte en lists la columna sobre los generos
    df_movies.loc[:,'listed_in'] = df_movies['listed_in'].apply(lambda x: x.split(', '))

    # Se creamn las columnas de tipo dummies para los generos
    from sklearn.preprocessing import MultiLabelBinarizer

    mlb = MultiLabelBinarizer()

    dummies_genres = pd.DataFrame(mlb.fit_transform(df_movies['listed_in']), columns=mlb.classes_)

    # Añadir las columnas al dataset
    df_movies = pd.concat([df_movies, dummies_genres], axis=1)

    # Convertir la columna de la fecha de adición en tipo DateTime de pandas
    series_fechas = pd.to_datetime(df_movies['date_added'].apply(lambda x: x.replace(',','')), format='%B %d %Y')

    df_movies['date_added'] = series_fechas.sort_values(ascending=True)

    # Finaliza el dataset
    df_movies.drop(columns=['type','director', 'cast', 'show_id','title','country', 'rating', 'duration', 'listed_in', 'description','release_year'], axis=1, inplace=True)
    df_movies.sort_values(by='date_added',inplace=True)
    df_movies.reset_index(drop=True, inplace=True)

    # Expresar las sumas cumuladas cada genero 
    df_movies_cumsum = df_movies.iloc[:,:].groupby('date_added').sum().cumsum()

    # Extraer la columna date_added desde el index
    df_movies_cumsum.reset_index(inplace=True)

    # Cambiar el "type" para las fechas en date_time de pandas
    df_movies_cumsum['date_added'] = pd.to_datetime(df_movies_cumsum['date_added'])

    # Grupacion de las fechas por el mese, es decir, reducir la longitud del dataset
    df_movies_cumsum['date_added'] = df_movies_cumsum['date_added'].dt.to_period('M') # Remover el dia desde la fecha
    df_movies_cumsum =df_movies_cumsum.iloc[:,:].groupby('date_added').max() # Tener solo el maximo alcanzado en tal mes
    df_movies_cumsum.reset_index(inplace=True)

    return df_movies_cumsum




In [136]:
def modifica_datos_series(df) :
    # Se divide entre datos sobre movies y datos sobre series tv
    df_series = df[df['type']=='TV Show']
    df_series.reset_index(drop=True, inplace=True)

    # Se convierte en lists la columna sobre los generos
    df_series.loc[:,'listed_in'] = df_series['listed_in'].apply(lambda x: x.split(', '))

    # Se creamn las columnas de tipo dummies para los generos
    from sklearn.preprocessing import MultiLabelBinarizer

    mlb = MultiLabelBinarizer()

    dummies_genres = pd.DataFrame(mlb.fit_transform(df_series['listed_in']), columns=mlb.classes_)

    # Añadir las columnas al dataset
    df_series = pd.concat([df_series, dummies_genres], axis=1)

    # # Convertir la columna de la fecha de adición en tipo DateTime de pandas
    # series_fechas = pd.to_datetime(df_series['date_added'].apply(lambda x: str(x).replace(',','')), format='%B %d %Y')

    df_series['date_added'] = series_fechas.sort_values(ascending=True)

    # Finaliza el dataset
    df_series.drop(columns=['type','director', 'cast', 'show_id','title','country', 'rating', 'duration', 'listed_in', 'description','release_year'], axis=1, inplace=True)
    df_series.sort_values(by='date_added',inplace=True)
    df_series.reset_index(drop=True, inplace=True)

    # Expresar las sumas cumuladas cada genero 
    df_series_cumsum = df_series.iloc[:,:].groupby('date_added').sum().cumsum()

    # Extraer la columna date_added desde el index
    df_series_cumsum.reset_index(inplace=True)

    # Cambiar el "type" para las fechas en date_time de pandas
    df_series_cumsum['date_added'] = pd.to_datetime(df_series_cumsum['date_added'])

    # Grupacion de las fechas por el mese, es decir, reducir la longitud del dataset
    df_series_cumsum['date_added'] = df_series_cumsum['date_added'].dt.to_period('M') # Remover el dia desde la fecha
    df_series_cumsum =df_series_cumsum.iloc[:,:].groupby('date_added').max() # Tener solo el maximo alcanzado en tal mes
    df_series_cumsum.reset_index(inplace=True)

    return df_series_cumsum

Comprobar los resultados

In [138]:
netflix_movies = modifica_datos_movies(df_netflix)
netflix_series = modifica_datos_series(df_netflix)

disney_movies = modifica_datos_movies(df_disney)
disney_series = modifica_datos_series(df_disney)

hulu_movies = modifica_datos_movies(df_hulu)
hulu_series = modifica_datos_series(df_hulu)

In [144]:
hulu_series

,date_added,Action,Adult Animation,Adventure,Anime,Black Stories,Cartoons,Classics,Comedy,Cooking & Food,...,Reality,Romance,Science & Technology,Science Fiction,Sitcom,Sketch Comedy,Sports,Stand Up,Teen,Thriller
0,2012-12,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2013-08,2,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2013-12,4,0,4,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2014-01,4,0,5,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2014-02,4,0,5,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2021-06,225,20,235,231,43,26,15,234,50,...,131,30,19,31,81,2,36,2,20,18
70,2021-07,233,21,241,234,46,26,15,254,61,...,166,38,20,35,87,2,36,2,24,20
71,2021-08,234,24,243,234,48,26,15,259,64,...,187,41,21,36,89,2,42,2,24,20
72,2021-09,238,24,249,238,52,27,15,270,69,...,205,45,22,38,94,2,49,2,25,22
